In [22]:
# Importing required Libraries
import csv
from bs4 import BeautifulSoup
from datetime import datetime 
import requests

In [23]:
# etting the url for our template
template= 'https://in.indeed.com/jobs?q={}&l={}'

In [51]:
def get_url(position,location):
    '''Generate an url for pos an loc'''
    template= 'https://in.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '%2C+')
    url=template.format(position,location)
    return url
url=get_url('python develepor','Bengaluru Karnataka')
print(url)
    

https://in.indeed.com/jobs?q=python+develepor&l=Bengaluru%2C+Karnataka


#### Extract Raw HTML

See here I have used another link url1 coz the one was causing sme trouble it was gving some missing links

In [28]:
url1='https://in.indeed.com/jobs?q=python+developer&l=Bengaluru%2C+Karnataka'
response=requests.get(url1)

In [29]:
# if <Response [200]>  is the o/p for this its working fine..
response 

<Response [200]>

In [30]:
soup=BeautifulSoup(response.text,'html.parser')

In [32]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')
len(cards)

15

#### Prototyping model with single record 

In [33]:
card=cards[0]

In [34]:
job_title = card.h2.a.get('title')

In [35]:
company = card.find('span', 'company').text.strip()

In [36]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')

In [37]:
post_date = card.find('span', 'date').text
today = datetime.today().strftime('%Y-%m-%d')

In [38]:
summary = card.find('div', 'summary').text.strip().replace('\n', ' ')

In [39]:
# this does not exists for all jobs, so handle the exceptions
salary_tag = card.find('span', 'salaryText')
if salary_tag:
    salary = salary_tag.text.strip()
else:
    salary = ''

In [40]:
job_url = 'https://www.indeed.com' + card.h2.a.get('href')

In [41]:
record = (job_title, company, job_location, post_date, today, summary, salary, job_url)

In [42]:
record

('Python/Django Developer',
 'Ernest Talent',
 'Bengaluru, Karnataka',
 '2 days ago',
 '2021-03-31',
 '4+ years of experience in Python Django development. Python django: 4 years (Required).',
 '',
 'https://www.indeed.com/company/Ernest-Talent/jobs/Python-Django-Developer-6bbcaecc2eed17d1?fccid=e4c10798223bc2cb&vjs=3')

#### Generalize the model with a function¶


In [43]:
def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
        
    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record

In [44]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)


#### Get the next page

In [45]:
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')

    for card in cards:
        record = get_record(card)
        records.append(record)

#### Put it all together

In [49]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = ('https://in.indeed.com/jobs?q=python+developer&l=Bengaluru%2C+Karnataka')
    return url


def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
        
    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
        
    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)

In [50]:
# run the main program
main('python developer', 'bengaluru karnataka')